In [1]:
!which python

/opt/homebrew/anaconda3/bin/python


In [2]:
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
import time

PINECONE_API_KEY = open("/Users/andre/startup/pinecone_api_key.txt", "r").read()
pc = Pinecone(api_key=PINECONE_API_KEY)
df = pd.read_csv("/Users/andre/startup/data/stanford_techfinder_10_pages.csv")

df.head()

,title,link,description
0,An Adjustable Mitral Ring for Asymmetric and C...,https://techfinder.stanford.edu/technology/adj...,Researchers at Stanford University have create...
1,Anatomically conforming device for tricuspid v...,https://techfinder.stanford.edu/technology/ana...,Stanford researchers have designed a novel tri...
2,Diode-pumped photonic integrated titanium-sapp...,https://techfinder.stanford.edu/technology/dio...,Stanford researchers have developed ultra-wide...
3,Wafer-Scale Thin-Film Titanium:Sapphire Photonics,https://techfinder.stanford.edu/technology/waf...,"Using their novel fabrication process, Stanfor..."
4,Hypoxia-mediated resistance of Homologous reco...,https://techfinder.stanford.edu/technology/hyp...,Stanford researcher's invivo studies find that...


In [8]:
index_name = "stanford-techfinder-10-v2"

# Already created
# pc.create_index(
#     name=index_name,
#     dimension=1024,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

In [16]:
#Convert DataFrame to list of dictionaries with id and text fields
data = [
    {
        "id": f"vec{i}",
        "text": f"{row.title}. {row.description}"  # Access attributes directly
    }
    for i, row in enumerate(df.itertuples(index=False))
]#[:96] # only take first 5 for testing

print(len(data))


150


In [15]:

embeddings = pc.inference.embed(
    model='multilingual-e5-large',
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)

print(embeddings[0])

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-10', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': 'd16cacf7947ed3b8f8dc9a3dc3c8a3ae', 'Date': 'Sat, 16 Nov 2024 20:35:55 GMT', 'Server': 'Google Frontend', 'Content-Length': '143', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Input length '150' exceeded inputs limit of 96 for model 'multilingual-e5-large'"},"status":400}


In [11]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)
    print("Waiting for index to be ready...")

index = pc.Index(index_name)

vectors = []
print(len(data), len(embeddings))
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })
print(len(vectors))

index.upsert(
    vectors=vectors,
    namespace="ns1"
)

print(index.describe_index_stats())

96 96
96
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [13]:
query = "How can I increase energy efficiency?"

embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

results = index.query(
    namespace="ns1",
    vector=embedding[0].values,
    top_k=5,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'vec93',
              'metadata': {'text': 'Increasing energy efficiency in robotic '
                                   'systems. Electric motors are widely used '
                                   'in robots but waste energy in many '
                                   'applications. This inefficiency leads to '
                                   'short battery life and hinders the '
                                   'adoption of new robotic technologies '
                                   'ranging from humanoids to exoskeletons. '
                                   'Researchers at Stanford have finally '
                                   'addressed this problem with a '
                                   'high-efficiency elastic energy-recycling '
                                   'actuator capable of reducing power '
                                   'consumption by 50-97%. These new actuators '
                                   'use controllable mechani